In [ ]:
import duckdb
import pandas as pd
con = duckdb.connect("/tmp/dbt.duckdb")

In [ ]:
table_schema = 'main_aadsc_data'
p = 'aadsc_ftd_participant'
c = 'aadsc_ftd_condition'
s = 'aadsc_ftd_study'

tables = [p,c,s]

## DB and Table general info

In [ ]:
# Fetch tables in the specified schema
t = con.execute(
    f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{table_schema}'"
).fetchall()
t

In [ ]:
# Fetch column names of the table
for table in tables:
    # if table == 'aadsc_ftd_participant':
    query = f"SELECT name FROM pragma_table_info('{table_schema}.{table}') LIMIT 15"
    column_names = con.execute(query).fetchall()
    print(f"column_names of table: {table}\n {column_names}")

## Sanity checks

In [ ]:
view_all = pd.DataFrame(con.execute(
        f"SELECT * FROM {table_schema}.{p} LIMIT 5"
    ).fetchall())
view_all

In [ ]:
view_all = pd.DataFrame(con.execute(
        f"SELECT * FROM {table_schema}.{c} LIMIT 5"
    ).fetchall())
view_all

In [ ]:
view_all = pd.DataFrame(con.execute(
        f"SELECT * FROM {table_schema}.{p} LIMIT 5"
    ).fetchall())
view_all

## Investigate columns

In [ ]:
# Count all rows 
count_all = con.execute(
    f"SELECT COUNT(*) FROM {table_schema}.{c}"
).fetchall()
print(f"\nCount n_rows: {count_all}")

# Count rows where missing mappings
count_null = con.execute(
    f"SELECT COUNT(*) as n_no_mapping FROM {table_schema}.{c} WHERE hpo_code IS NULL AND mondo_code IS NULL AND other_code IS NULL"
).fetchall()
print(f"\nERROR: Count of rows where MISSING mapping: {count_null}")

#    Count rows where there are mappings
count_not_null = con.execute(
    f"SELECT COUNT(*) as n_have_mapping FROM {table_schema}.{c} WHERE hpo_code IS NOT NULL OR mondo_code IS NOT NULL OR other_code IS NOT NULL"
).fetchall()
print(f"\nCount of rows where one column is NOT NULL: {count_not_null}")


In [ ]:
dist_cond = con.execute(
f"SELECT distinct ethnicity FROM {table_schema}.{p}"
).fetchall()
print(f"\nView distinct ethnicities. {dist_cond}")

## Notes 

Copy results to a csv <br>
    ```
        to_csv = con.execute( 
        f"COPY (SELECT DISTINCT cardiac_pfo_age,cardiac_pda_age,skin_acne_age,skin_alopecia_age,cardiac_asd_age,cardiac_tof_age,cardiac_vsd_age FROM {table_schema}.{table}) TO 'output_src.csv' (HEADER, DELIMITER ',')"
    ).fetchall()
    ```